1. Install the Gen AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [ ]:
!pip install --upgrade google-genai
!export GOOGLE_CLOUD_API_KEY="YOUR_API_KEY"

2. Use the following code in your application to request a model response

In [ ]:
from google import genai
from google.genai import types
import base64
import os

def generate():
  client = genai.Client(
      vertexai=True,
      api_key=os.environ.get("GOOGLE_CLOUD_API_KEY"),
  )

  msg1_image1 = types.Part.from_uri(
      file_uri="gs://qwiklabs-gcp-00-7da84e16132a-bucket/cymbal-product-image.png",
      mime_type="image/png",
  )
  msg2_text1 = types.Part.from_text(text="""**Considering Image Composition**

I've isolated the key visual elements: hiking trail, wildflowers, map, backpack, and water bottle. Now, I'm working to integrate these into a single, cohesive sentence that encapsulates both the scene and hints at the product's use. It's a tricky balance of descriptive detail and conciseness, especially with the user's expertise in mind.""")
  msg2_text2 = types.Part.from_text(text="""A blue hiking backpack with a teal water bottle in its side pocket sits beside a winding trail through a vibrant mountain meadow filled with colorful wildflowers, with a map unfolded nearby.""")
  msg4_text1 = types.Part.from_text(text="""**Brainstorming Taglines**

My focus is on crafting a catchy tagline for an outdoor gear ad. The image inspires me, and I'm playing with words related to adventure and nature. I've considered journey and preparedness as key themes. Currently, I'm leaning toward short, evocative phrases that capture the essence of exploration.


**Developing The Concept**

I've been honing in on the tagline \"Unfold your adventure,\" it aligns with the map imagery, and suggests a personal journey of discovery. I've also liked \"Where every trail blooms with possibility,\" linking to the visual of wildflowers. I'm aiming for brevity and impact, playing with word choice to evoke both the practical and emotional aspects of the image. My current focus is refining those options further.


**Formulating The Perfect Slogan**

I'm now focused on the final selection, going back and forth between \"Your Path. Your Pack. Your Adventure,\" and a new iteration: \"Find Your Path. Pack Your Adventure.\" I've considered the flow and rhythm of the phrases, wanting the words to linger in the mind. The goal is to encapsulate the excitement and promise of exploration, directly tied to the gear itself.


**Evaluating Iterations**

I'm now refining the options, specifically focusing on how the taglines resonate with the image. \"Unfold Your Next Adventure\" feels strong, and I like how it directly ties to the map. Also, I'm analyzing the feeling of starting a new journey, and the appeal of the beauty of nature. My attention has turned to the flow and simplicity of the language, to create something that sparks instant interest.


**Synthesizing The Ideas**

I'm now streamlining the chosen tagline, still favoring phrases with \"Unfold\" given the map imagery. I've narrowed down to a few options: \"Unfold Your Next Adventure\" and \"Adventure Unfolded.\" I'm also considering the tone I want to convey, the imagery of nature, and the implied preparation. The final decision depends on how effectively each option captures the blend of exploration and readiness for adventure.


**Narrowing The Field**

I'm now revisiting options, specifically focusing on the most promising taglines. The simple power of \"Unfold Your Adventure\" still resonates, as does \"Adventure Unfolded,\" both playing off the map imagery. I'm also considering the tone, trying to evoke a sense of freedom and readiness. My goal is to select the perfect balance of brevity and impact.""")
  msg6_text1 = types.Part.from_text(text="""**Evoking a Scene's Essence**

I'm presently focusing on how to translate the visual elements into evocative language. The goal is to capture the feelings evoked by the trail, the mountainside, and the wildflowers. I'm exploring different word choices to emphasize the vibrancy of the colors and the overall feeling of serenity, considering phrases to convey a sense of gentle movement and the journey ahead. The backpack and water bottle seem like tangible elements of a story, too.


**Imagining a Narrative's Start**

Now I'm envisioning the scene as the opening of a story. I'm focusing on the trail's path and the map as catalysts for adventure, emphasizing the contrast between the stillness of the moment and the anticipation of movement. The sunlight and the wildflowers add layers of sensory detail to the scene. The backpack and water bottle are a great source to ground the imagery, giving a familiar and inviting aspect to the unknown trail ahead.""")
  msg6_text2 = types.Part.from_text(text="""Sun-kissed slopes unfold, a tapestry of green and gold,
Where ancient pathways gently wind, leaving the world's dull cares behind.
A symphony of hues, where sapphire bells and buttercup's blaze,
Dance in the alpine breeze, through sun-drenched, tranquil days.
A silent invitation lies, within the map's soft, whispered plea,
To shed all burdens, light and free, and find the soul's wild ecstasy.
The pack awaits, a faithful guide, to journeys yet untold,
Where every bloom and mountain high, a story will unfold.
It's solace found, where silence sings, and nature's spirit gently brings,
A peace profound, an open heart, a brand new, hopeful start.""")

  model = "gemini-2.5-flash"
  contents = [
    types.Content(
      role="user",
      parts=[
        msg1_image1,
        types.Part.from_text(text="""Describe this outdoor product image in one sentence.""")
      ]
    ),
    types.Content(
      role="model",
      parts=[
        msg2_text1,
        msg2_text2
      ]
    ),
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Generate a catchy tagline suitable for an outdoor gear advertisement based on this image.""")
      ]
    ),
    types.Content(
      role="model",
      parts=[
        msg4_text1,
        types.Part.from_text(text="""Unfold Your Adventure.""")
      ]
    ),
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Write a poetic description inspired by this outdoor scene. Focus on mood and feeling.""")
      ]
    ),
    types.Content(
      role="model",
      parts=[
        msg6_text1,
        msg6_text2
      ]
    ),
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Make the tone more energetic.""")
      ]
    ),
  ]
  tools = [
    types.Tool(google_search=types.GoogleSearch()),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    tools = tools,
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
        continue
    print(chunk.text, end="")

generate()